In [4]:
import pickle
import numpy as np
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import pandas as pd
import html2text
import requests
from datetime import datetime
import re
#nltk.download('stopwords')

In [761]:
from pymongo import MongoClient
#CEDC_Quigley
# lpP9pCcDxQ2Y0tv8

client = MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')

database = client['legislative_documents']
collection = database['legiscan_bill_metadata']

legtext_corpus = [[0,0,['dummy','entry']]]

title =[]
bill_id=[]
body=[]
month_intro=[]
year_intro=[]
day_intro=[]
state_link=[]
sponsor_id=[]


for bl in collection.find({'state':"NH"}).batch_size(30):
    bill_id.append(str(bl['bill_id']))
  #  state_link.append(str(bl['']))
    title.append(str(bl['title']))
    
    body.append(str(bl['body']))

    bill_time_stamp = str(bl['status_date'])
    
    if bill_time_stamp == "None":
        _time= -1
        year_intro.append(0)
        month_intro.append(0)
    else:
        _time = datetime.strptime(bill_time_stamp, "%Y-%m-%d")
        year_intro.append(int(_time.strftime("%Y")))
        month_intro.append(int(_time.strftime("%m")))
        day_intro.append(int(_time.strftime('%d')))

In [10]:
#sponsor data
sponsor_id=[]
sponsor_name=[]
sponsor_party=[]
sponsor_order=[]
sponsor_type_id=[]
committee_id=[]

for i, bl in enumerate(collection.find({'state':"NH"}).batch_size(30)):
    sponsor_id.append([])
    sponsor_name.append([])
    sponsor_party.append([])
    sponsor_order.append([])
    sponsor_type_id.append([])
    committee_id.append([])
    for b in bl['sponsors']:
        sponsor_id[i].append(str(b['people_id']))
        sponsor_name[i].append(str(b['name']))
        sponsor_party[i].append(str(b['party']))
        sponsor_order[i].append(str(b['sponsor_order']))
        sponsor_type_id[i].append(str(b['sponsor_type_id']))
        committee_id[i].append(str(b['committee_id']))

In [ ]:
sponsor_dict ={'bill_id': bill_id
               ,'sponsor_id': sponsor_id
               ,'sponsor_party': sponsor_party
               ,'sponsor_order': sponsor_order
               ,'sponsor_type_id': sponsor_type_id
              ,'sponsor_name': sponsor_name}

In [11]:
#co-sponsor info
sponsor_party_vec=[] #average affiliation
sponsor_party_maj=[] #1 or 0 if the overall average is in the majority
num_maj=[] #the number of co sponsors in the majority
x=0
for i in sponsor_party:
    for j in range(len(i)):
        if i[j]=='R':
            x+=2
        if i[j]=='D':
            x+=1
        else:
            x=x
    if len(i)!=0:
        x=x/ len(i)
    sponsor_party_vec.append(x)  
    x=0

In [1545]:
print(sponsor_party_vec[0:100], sponsor_party[0:10])

[2.0, 0.9, 1.0, 2.0, 1.6666666666666667, 2.0, 2.0, 2.0, 1.1428571428571428, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.75, 2.0, 2.0, 1.8333333333333333, 2.0, 1.6666666666666667, 1.25, 2.0, 2.0, 1.0, 2.0, 1.3333333333333333, 2.0, 2.0, 2.0, 1.5, 1.0, 1.0, 2.0, 2.0, 2.0, 1.8333333333333333, 2.0, 2.0, 1.4, 1.3, 2.0, 2.0, 1.8, 2.0, 2.0, 1.6666666666666667, 1.3333333333333333, 1.8, 2.0, 2.0, 1.25, 1.0, 2.0, 1.0, 1.3333333333333333, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.6666666666666667, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.5, 1.8888888888888888, 1.0, 1.0, 2.0, 2.0, 1.5, 2.0, 2.0] [['R'], ['D', 'D', 'D', 'I', 'D', 'D', 'D', 'D', 'D', 'D'], ['D'], ['R'], ['R', 'D', 'R', 'R', 'D', 'R'], ['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R'], ['R', 'R', 'R', 'R', 'R'], ['R', 'R'], ['D', 'D', 'D', 'D', 'D', 'R', 'D'], ['D']]


In [776]:
num_maj=[-1]*len(year_intro)
rep=0
dem=0
for j in range(len(year_intro)):
    if year_intro[j] in year:
        for h in range(len(year)):
            if H_majority[h]>1.5:
                for cosponsors in sponsor_party:
                    for k in cosposnors:


In [777]:
print(num_maj[0:10])

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [515]:
#committee data-- missing for alot of them
committee=[]
for bl in (collection.find({'state':"NH"}).batch_size(30)):
    committee.append(str(bl['committee']))
    
#doesnt work
committee_intro=[]
for i in range(len(committee)-1):
    text=committee[i].split()
    committee_=text[len(text)-1]
    committee_=committee_[:-1] 
    committee_intro.append(committee_)

In [517]:
# alternative to getting which committee, is using the first one in progress
history_=[]
for bl in (collection.find({'state':"NH"}).batch_size(30)):
        history_.append(str(bl['history']))

In [290]:
dates=[]
actions=[]
for i, bl in enumerate(collection.find({'state':"NH"}).batch_size(30)):
    dates.append([])
    actions.append([])
    for b in bl['history']:
        dates[i].append(str(b['date']))
        actions[i].append(str(b['action']))

In [291]:
last_date=[]
intro_date=[]
year_passed=[]
month_passed=[]

x=0
for i in range(len(dates)):
    if dates[x]!=[]:
        intro_date.append((dates[x][0]))
        last_date.append((dates[x][len(dates[x])-1]))
    else:
        intro_date.append(0)
        last_date.append(0)
    x+=1

In [14]:
#average time between steps
avg_time=[0]*len(dates)
temp=[]
for i in range(len(dates)):
    if dates[x]!=[]:
        for j in range(len(dates[x]))
            temp.append(datetime.strptime(dates[i][j], "%Y-%m-%d"))
            tempyr.append(int(temp[i].strftime("%Y")))
            tempmon.append(int(temp[i].strftime("%m")))

SyntaxError: invalid syntax (<ipython-input-14-78a409fa8808>, line 6)

In [ ]:
#finds how many months in session
year_passed=[]
month_passed=[]
_last_date=[]

for i in range(len(last_date)):
    if last_date[i]!=0:
        _last_date.append(datetime.strptime(last_date[i],"%Y-%m-%d"))
    elif last_date[i]==[]:
        _last_date.append(0)
    else:
        _last_date.append(0)
        
for i in range(len(_last_date)):
    if _last_date[i]!=0:
        year_passed.append(int(_last_date[i].strftime("%Y")))
        month_passed.append(int(_last_date[i].strftime("%m")))
    else:
        year_passed.append(0)
        month_passed.append(0)
        
num_years=[0]*len(year_passed)
for i in range(len(year_passed)):
    if year_passed[i]!=0 and year_intro[i]!=0:
        num_years[i]=abs(year_passed[i]-year_intro[i])
    elif abs(year_passed[i]-year_intro[i]>=2000):
        num_years[i]=-1
    else:
        num_years[i]=-1
        
time_session=[0]*len(month_passed)
for i in range(len(month_passed)):
    if month_passed[i]!=0 and year_intro[i]!=0:
        time_session[i]=abs(month_passed[i]-month_intro[i]) + 12*num_years[i]
    else:
        time_session[i]=-1

In [564]:
first_action=[]
for i in range(len(actions)):
    if actions[i]!=[]:
        first_action.append((actions[i][0]))
    else:
        first_action.append("None")

#instead getting committee from the first action
committee_=["None"]*len(first_action)
for i in range(len(actions)):
    text=first_action[i].split()
    for j in range(len(text)):
        if text[j]=="to" and text[j-1]=="Referred":
            committee_[i]=(text[j+1:j+7])
            
committee_intro=[]
for i in committee_:
    if "House" in i[0:len(i)]:
        y=i[0:len(i)].index("House")
        i=i[0:y]
        committee_intro.append(i)
    elif "Senate" in i[0:len(i)]:
        y=i[0:len(i)].index("Senate")
        i=i[0:y]
        committee_intro.append(i)
    else:
        i=i
        committee_intro.append(i)
#clean up committee_intro, get rid of any upper case, or special characters
temp=" "
committee_intro2=["None"]* len(committee_intro)
for i in range(len(committee_intro)):
    committee_intro2[i]=(temp.join(committee_intro[i]))
    temp=" "

com_intro=[]  
for i in range(len(committee_intro2)):
     com_intro.append(committee_intro2[i].replace(";",'').lower(). replace("&", "and"). replace(",",''))

In [566]:
print(com_intro[56])

science technology and energy


In [16]:
#primary sponsor info
primary_id=[]
primary_name=[]
primary_party=[]

x=0
for j in range(len(sponsor_name)):
    if sponsor_name[x]!=[]:
        primary_name.append((sponsor_name[x][0]))
    else:
        primary_name.append("None")
    x+=1
x=0
for j in range(len(sponsor_id)):
    if sponsor_id[x]!=[]:
        primary_id.append(int(sponsor_id[x][0]))
    else:
        primary_id.append(0)
    x+=1
x=0
for j in range(len(sponsor_party)):
    if sponsor_party[x]!=[]:
        primary_party.append((sponsor_party[x][0]))
    else:
        primary_party.append("None")
    x+=1

In [ ]:
#number of cosponsors
num_cosponsors=[]
x=0
for j in range(len(sponsor_id)):
    num_cosponsors.append(len(sponsor_id[x]))
    x+=1

In [ ]:
#number of steps/stages
num_stages=[]
x=0
for j in range(len(actions)):
    num_stages.append(len(actions[x]))
    x+=1

In [302]:
#binary is primary sponsor on committee-- missing data
committe_prim=[]
x=0
for j in range(len(committee_id)):
    if committee_id[x]!=[]:
        committe_prim.append(committee_id[x][0])
    else:
        committe_prim.append("None")
    x+=1
#binary is any sponsor on committee-  missing data
committe_sponsor=[]
for j in range(len(committee_id)):
    if committee_id[x]!=[]:
        committe_prim.append(committee_id[x][0])
    else:
        committe_prim.append("None")
    x+=1

In [17]:
## this prints the ids of the senate legislators at the senate in 2015
with open('politicians_by_year.dict', 'rb') as f:
    politicians=pickle.load(f)
      

#the number of terms a sponsor served        
year=2019
year_min=2014
body_= ['H', 'S']
state='NH'
ids=[]
party=[]

while year > year_min :  
    x= len((politicians[state][year]['H']))
    for i in range (x):
        ids.append(politicians[state][year]['H'][i])
    year-=1

while year > year_min :  
    x= len((politicians[state][year]['S']))
    for i in range (x):
        ids.append(politicians[state][year]['S'][i])
    year-=1 

# num_terms is the number of terms a sponsor served
temp=[]
mul_term=[]
num_term=[]
for j in range(len(ids)):
    temp.append([i for i, value in enumerate(ids) if value ==(ids[j])] )
    
len_term=2                         
for i in range(len(temp)):
    for j in range(len(temp[i])):
        mul_term.append(ids[temp[i][j]])
    num_term.append(len(temp[i])/len_term)

mul_term2=[]
for i in range(len(mul_term)):
    if mul_term[i-1]!=mul_term[i]:
        mul_term2.append(mul_term[i])
        
num_terms=[-1]* len(primary_id) 
for j in range(len(primary_id)):
    if primary_id[j] in mul_term2:
        temp = [i for i, value in enumerate(primary_id) if value== primary_id[j]]
        for i in range(len(temp)):
            num_terms[temp[i]]=(num_term[i])  

In [18]:
with open('people_party_table.dict', 'rb') as f:
    politicians_party= pickle.load(f)

In [19]:
with open('people_info_table.dict', 'rb') as f:
    people_table= pickle.load(f)

In [763]:
x=0
year=[2015,2016,2017,2018,2019]
state=['NH']
H_majority=[]
S_majority=[]
y=0

for j in year:   
    for i in politicians_party["NH"][j]["S"]:
        if politicians_party["NH"][j]["S"][i] == '1':
            x+=1
        if politicians_party["NH"][j]["S"][i] == '2':
            x+=2
        y=x/len(politicians_party["NH"][j]["S"])
    S_majority.append(y)
    x=0
    y=0
    for i in politicians_party["NH"][j]["H"]:
        if politicians_party["NH"][j]["H"][i] == '1':
            x+=1
        if politicians_party["NH"][j]["H"][i] == '2':
            x+=2
        y=x/len(politicians_party["NH"][j]["H"])
    H_majority.append(y)
    x=0
    y=0

j=0
h=0
maj_party=[-1]* len(primary_party)
for j in range(len(year_intro)):
    for h in range(len(year)):
        if year_intro[j]==year[h] and body[j]=='H':
            if H_majority[h]>1.5 and primary_party[j]=='R':
                maj_party[j]=1
            else:
                maj_party[j]=0

            if H_majority[h]==1.5: #equal party distribution, zero for everyone
                maj_party[j]=0

            if H_majority[h]<1.5 and primary_party[j]=='D':
                maj_party[j]=1
                
        if year_intro[j]==year[h] and body[j]=='S':
            if S_majority[h]>1.5 and primary_party[j]=='R':
                maj_party[j]=1
            else:
                maj_party[j]=0

            if S_majority[h]==1.5: #equal party distribution, zero for everyone
                maj_party[j]=0

            if S_majority[h]<1.5 and primary_party[j]=='D':
                maj_party[j]=1
                
#important note: here i used body as an assumption that if the primary is in the hosue the bill is introcued
#there, this is not alwasy true, but since for NH the majority R or D is the same for the same years
#this assumption is fine, we need to go back and fix this so that it is based off on chamber

In [762]:
print(len(body))

10613


In [ ]:
##### time from start and end of time session, manually build a variable for NH, for all states refer to:
#https://www.ncsl.org/research/about-state-legislatures/2009-legislative-session-calendar.aspx

year = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
start_date= ['2009-01-07', '2010-01-06', '2011-01-05', '2012-01-04', '2013-01-02', '2014-01-08', '2015-01-07', '2016-01-06', '2017-01-04', '2018-01-03', '2019-01-02']
end_date= ['2009-07-01', '2010-07-01', '2011-07-01', '2012-06-27', '2013-07-01', '2014-06-13', '2015-07-01', '2016-06-01', '2017-06-22', '2018-06-30', '2019-06-30']


start_month=[]
start_year=[]
start_day=[]

end_month=[]
end_year=[]
end_day=[]


for i in range(len(start_date)):
        start_date[i]= datetime.strptime(start_date[i], "%Y-%m-%d")
        end_date [i]= datetime.strptime(end_date[i], "%Y-%m-%d")
       
        start_year.append(int(start_date[i].strftime("%Y")))
        end_year.append(int(end_date[i].strftime("%Y")))
        
        start_month.append(int(start_date[i].strftime("%m")))
        end_month.append(int(end_date[i].strftime("%m")))
        
        start_day.append(int(start_date[i].strftime('%d')))
        end_day.append(int(start_date[i].strftime('%d')))
        

NH_cal_dict={'year': year,
            'leg session start': start_date,
            ' leg session end': end_date}


In [1640]:
_month_session=[]
_day_session=[]

for i in range(len(year_intro)):
    if year_intro[i]==2009:
        _month_session.append(month_intro[i]-start_month[0])
        _day_session.append(start_day[0])
    elif year_intro[i]==2010:
        _month_session.append(month_intro[i]-start_month[1])
        _day_session.append(start_day[1])
    elif year_intro[i]==2011:
        _month_session.append(month_intro[i]-start_month[2])
        _day_session.append(start_day[2])
    elif year_intro[i]==2012:
        _month_session.append(month_intro[i]-start_month[3])
        _day_session.append(start_day[3])
    elif year_intro[i]==2013:
        _month_session.append(month_intro[i]-start_month[4])
        _day_session.append(start_day[4])
    elif year_intro[i]==2014:
        _month_session.append(month_intro[i]-start_month[5])
        _day_session.append(start_day[5])
    elif year_intro[i]==2015:
        _month_session.append(month_intro[i]-start_month[6])
        _day_session.append(start_day[6])
    elif year_intro[i]==2016:
        _month_session.append(month_intro[i]-start_month[7])
        _day_session.append(start_day[7])
    elif year_intro[i]==2017:
        _month_session.append(month_intro[i]-start_month[8])
        _day_session.append(start_day[8])
    elif year_intro[i]==2018:
        _month_session.append(month_intro[i]-start_month[9])
        _day_session.append(start_day[9])
    elif year_intro[i]==2019:
        _month_session.append(month_intro[i]-start_month[10])
        _day_session.append(start_day[10])
    else:
        _month_session.append(-20)
        _day_session.append(-20)

for i in range(len(_month_session)):
    if _month_session[i]==0:
        temp=abs((day_intro[i] - _day_session[i]))/30
        _month_session[i]=temp   

In [25]:
print((committee_table['Jordan_Ulery']))

{'year': ['2017', '2015', '2013-2014', '2011-2012'], 'committee/positions': [' ways and means/member', ' ways and means/member', ' ways and means/member', ' ways and means/member']}


In [738]:
#brings in the committee data
with open('/home/ec2-user/SageMaker/ProjectQuigley/sponsor_comittee_table.dict', 'rb') as f:
    committee_table=pickle.load(f)

#cleaning up the committee data
dictlist=[]
for key, value in committee_table.items():
    temp = [key,value]
    dictlist.append(temp)

name=[]
year_committee=[]
for i in range(len(dictlist)):
    name.append(dictlist[i][0])
    year_committee.append(dictlist[i][1])
    
year_com=[]
com_pos=[]
for i in year_committee:
    year_com.append(i['year'])
    com_pos.append(i['committee/positions'])

for i in range(len(name)):
    name[i]=name[i].replace("_", " ")  
    
year_comm=[]
com_pos=[]
for i in year_committee:
    year_comm.append(i['year'])
    com_pos.append(i['committee/positions'])
    
position=[]
comm_served=[]
for i in range(len(com_pos)):
    position.append([])
    comm_served.append([])
    for j in range(len(com_pos[i])):
        if "/" in com_pos[i][j]:
            position[i].append(com_pos[i][j][(com_pos[i][j].index("/")+1):])
            comm_served[i].append(com_pos[i][j][:(com_pos[i][j].index("/"))].strip())
            
year_com2=[]
for i in (range(len(year_comm))):
    year_com2.append([])
    for j in range(len(year_comm[i])):     
        if "-" in year_comm[i][j]:
            year_com2[i].append(int(year_comm[i][j][:4]))
            year_com2[i].append(int(year_comm[i][j][5:]))
        else:
            year_com2[i].append(int(year_comm[i][j]))
            
position2=[]
for j in range(len(year_com2)):
    for i in range(len(position)):
        if len(year_com2[j])!=len(position[j]):
            position[j].append(position[j])
            
primary_on_comm=[0]*len(primary_name)
pos_on_com=[0]*len(primary_name)

for i in range(len(primary_name)):
    if primary_name[i] in name:
        x= name.index(primary_name[i])
        if com_intro[i] in comm_served[x]:
            if year_intro[i] in year_com2[x]:
                primary_on_comm[i]=1 
                h= year_com2[x].index(year_intro[i])
                pos_on_com[i]= position[x][h]


In [1505]:
new_features_dict ={'bill_id':  bill_id
                    ,'title': title
                    ,'house/sentate': body
                    , 'primary name': primary_name
                    , 'primary id': primary_id
                    , 'primary party': primary_party
                    , 'number of co sponsors': num_cosponsors
                    , "committee introduced": committee_intro
                    , "number of terms served": num_terms
                    , 'primary in majority party': maj_party
                    , 'primary is on the committee': primary_on_comm
                    , 'position of primary on committee': pos_on_com
                    , 'month introduced': month_intro
                    , 'year introduced': year_intro
                    , 'month passed': month_passed
                    , 'year passed': year_passed
                    , "months from the start of legislative calender intro": _month_session}

In [ ]:
# Features to look at: 

#total of Democrats, total republicans 
# if sponsor is in majority binary
# number of cosponsors in majority (int)
# how many months it stayed
# average leadership score
# total leadership score of all sponsors
#leadership score for the primary
# average time between steps 
# time since the start of the session

#poltician features

# poltician of each year, what i want to do is figure out the following:
# how many terms a sponsor spent
# what is the majority party of the house/senate for a given year
# is the primary sponsor in the majority of the part


In [1501]:
print(len(bill_id),len(title),len(body),len(primary_name), len(primary_id) , len(num_terms), len(maj_party))

10613 10613 10613 10613 10613 10613 10613


In [1504]:
print(len(primary_party), len(committee_intro), len(num_stages))

10613 10613 10613


In [1508]:
#creating the target dataset for overall classifiers leg_win_state.csv
bill_features = pd.DataFrame(new_features_dict)

# creating an overall dataset with defined win condition
with open('BillFeatures.dict','wb') as f:
    pickle.dump(bill_features, f)

In [1509]:
bill_features.to_csv('bill_features.csv')